In [14]:
# Load calibration data
import pickle
with open('./pickel_files/all_data_200_100.pkl', 'rb') as f:
    X, Y = pickle.load(f)

In [16]:
from keras.models import load_model
import numpy as np
import cv2



# Initialize your gaze detection model
model = load_model('./eye_gaze_v23_1_10.h5')

# Predict gaze points for each image in the calibration dataset
predicted_gaze_points = []
for image in X:
    image_reshaped = np.expand_dims(image, axis=0)
    predicted_point = model.predict(image_reshaped)  
    predicted_gaze_points.append(predicted_point)

# Create the dataset for the adjustment model
adjustment_dataset = {
    'predicted_gaze_points': predicted_gaze_points,
    'actual_gaze_points': Y
}

# Save the dataset into a new pickle file
with open('adjustment_dataset.pkl', 'wb') as f:
    pickle.dump(adjustment_dataset, f)


1/1 [==============================] - 0s 42ms/step


In [17]:
with open('./adjustment_dataset.pkl', 'rb') as f:
    data = pickle.load(f)
x= data['predicted_gaze_points']
x = np.squeeze(np.array(x))
x.shape

(756, 2)

In [20]:
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

# Load your data
with open('./adjustment_dataset.pkl', 'rb') as f:
    calibration_data = pickle.load(f)


X = np.squeeze(np.array(calibration_data['predicted_gaze_points']), axis=1)  # Remove extra dimensions
y = np.array(calibration_data['actual_gaze_points'])[:, :2]  # Remove extra dimensions
=
# X = calibration_data['predicted_gaze_points']  # Predicted gaze points from the main model
# y = calibration_data['actual_gaze_points']  # Actual gaze points

print(X.shape)
print(y.shape)

# Split the dataset
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=None)


# Initialize and train the model
model = RandomForestRegressor(n_estimators=1000, random_state=None)
model.fit(X_train, y_train)

# Predict on the validation set
y_pred = model.predict(X_val)

# Evaluate the model
mse = mean_squared_error(y_val, y_pred)
print(f'Mean Squared Error: {mse}')



(756, 2)
(756, 2)
Mean Squared Error: 0.019357543049932546


c:\Users\wgoud\.conda\envs\tf2.10_eye_gaze\lib\site-packages\sklearn\metrics\_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_true = check_array(y_true, ensure_2d=False, dtype=dtype)


In [21]:
# Save the model
with open('adjustment_model.pkl', 'wb') as f:
    pickle.dump(model, f)